# This sample was inspired by [08-native-function-inline.ipynb](https://github.com/microsoft/semantic-kernel/blob/main/python/notebooks/08-native-function-inline.ipynb)

## [How does Python SK compare to the C# version of Semantic Kernel?](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)
- The two SDKs are compatible and at the core they follow the same design principles.
- Some features are still available only in the C# version, and being ported.
- Refer to the [FEATURE MATRIX](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) doc to see where things stand in matching the features and functionality of the main SK branch.
- Over time there will be some features available only in the Python version, and others only in the C# version, for example adapters to external services, scientific libraries, etc.
<br/>
- Documentation
 - [Get Started with Semantic Kernel](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)

Inspired to
 - https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example59_OpenAIFunctionCalling.cs
 - https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

In [1]:
# Create "Native" (aka "Core") Plugins that contain "Native" Functions

from semantic_kernel import KernelContext
from semantic_kernel.plugin_definition import kernel_function, kernel_function_context_parameter



# --------------- CatClass

class CatClass:
    """
    Description: Given a cat name, returns some information.
    """    
    @kernel_function(
        description="Given a cat name, returns its age.",
        name="CatAge",
    )
    @kernel_function_context_parameter(name="cat_name", description="The cat name", default_value="AA")
    def cat_age(self, cat_name: str = "AA") -> str:
        """
        Given a cat name, returns its age.
        """
        # print(f"cat_name: {cat_name}")
        try:
            
            return str(len(cat_name))
        except ValueError as e:
            print(f"Invalid input {cat_name}")
            raise e
            
    @kernel_function(
        description="Given a cat name, returns its birthday.",
        name="CatBirthday",
    )
    @kernel_function_context_parameter(name="cat_name", description="The cat name", default_value="AA")
    def cat_birthday(self, cat_name: str = "AA") -> str:
        """
        Given a cat name, returns its birthday.
        """
        # print(f"cat_name: {cat_name}")
        from datetime import datetime
        from dateutil.relativedelta import relativedelta
        
        today_str  = datetime.today().strftime("%Y-%m-%d")
        today_date = datetime.strptime(today_str,"%Y-%m-%d")

        try:
            cat_birthday_str = today_date.strftime(f"{2000+len(cat_name)}-%m-%d")
            cat_birthday_date = datetime.strptime(cat_birthday_str,"%Y-%m-%d") # error raised when 29/2 in non-leap years
        except ValueError as e:
            cat_birthday_str = (today_date + relativedelta(days=1)).strftime(f"{2000+len(cat_name)}-%m-%d") # take March 1st

        return cat_birthday_str

    
# --------------- RandomNumberClass

    
class RandomNumberClass:
    """
    Description: Generate some random numbers.
    """

    @kernel_function(
        description="Generate a random number between 3 and x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3 and <input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        import random

        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

    @kernel_function(
        description="Generate a random number between x and three",
        name="GenerateNumberThreeOrLower",
    )
    def generate_number_three_or_lower(self, input: str) -> str:
        """
        Generate a number between <input> and 3
        Example:
            "-4" => rand(-4,3)
        Args:
            input -- The lower limit for the random number generation
        Returns:
            int value
        """
        import random
        
        try:
            return str(random.randint(int(input),3))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

In [2]:
# test the classes (NOT the plugin!)

my_cat_object = CatClass()  # Create an instance
my_cat_age = my_cat_object.cat_age() # Call the method
print(f"my_cat_age: {my_cat_age}") # Assert the result

print (my_cat_object.cat_age("Molly"))
print (my_cat_object.cat_birthday("Molly"))

print(RandomNumberClass().generate_number_three_or_lower(-30))

my_cat_age: 2
5
2005-02-07
-28


# Create the Semantic Kernel to load classes as plugins into it

In [3]:
import semantic_kernel as sk
kernel = sk.Kernel()

In [4]:
# import plugins into the kernel, together with all their (native) functions
kernel.import_plugin(plugin_instance=CatClass(), plugin_name="CatPlugin")
kernel.import_plugin(plugin_instance=RandomNumberClass(), plugin_name="RandomNumberPlugin")
print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

Here are my plugins imported into the kernel:
data={'catplugin': {'catage': KernelFunction(), 'catbirthday': KernelFunction()}, 'randomnumberplugin': {'generatenumberthreeorhigher': KernelFunction(), 'generatenumberthreeorlower': KernelFunction()}}


# Call Native Functions using plugins imported into the Kernel
## Note that we don't need any connection to Open AI to do this

In [5]:
# test plugin functions after extracting them from the kernel

print(kernel.plugins.get_function(plugin_name="RandomNumberPlugin", function_name="GenerateNumberThreeOrHigher")("20"))
print(kernel.plugins.get_function(plugin_name="RandomNumberPlugin", function_name="GenerateNumberThreeOrLower")("-20"))
print(kernel.plugins.get_function(plugin_name="CatPlugin", function_name="CatBirthday")("Tom"))
print(kernel.plugins.get_function(plugin_name="CatPlugin", function_name="CatBirthday")("Cleopatra"))
print(kernel.plugins.get_function(plugin_name="CatPlugin", function_name="CatAge")("Cleopatra"))
print(kernel.plugins.get_function(plugin_name="CatPlugin", function_name="CatAge")("Molly"))

8
-20
2003-02-07
2009-02-07
9
5


In [6]:
# another way to test Cats_Birthday function after extracting it from the kenel

cb_function = kernel.plugins.get_function(plugin_name="CatPlugin", function_name="CatBirthday")
answer = cb_function.invoke("Molly")
print (answer)

2005-02-07


## Add Open AI Chat Completion "Connector" to allow Kernel call Semantic Functions.
### Otherwise we get the error `<TextCompletionClientBase service with service_id 'None' not found>`

In [7]:
# We haven't used OpenAI yet
# So we create the SK "chat completion connector" to the Azure OpenAI service

from dotenv import load_dotenv
load_dotenv("credentials_my.env")

import semantic_kernel.connectors.ai.open_ai as sk_oai

chat_connector = sk_oai.AzureChatCompletion(
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    deployment_name=os.environ['GPT35TURBO-0613-4k'], # ['GPT4-1106-128k'],
    endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)
print(f"This is my AzureChatCompletion connector:\n{chat_connector}")

This is my AzureChatCompletion connector:
ai_model_id='gpt35turbo-0613-4k' client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7fa6e0f2f980> ai_model_type=<OpenAIModelTypes.CHAT: 'chat'> prompt_tokens=0 completion_tokens=0 total_tokens=0


In [8]:
# add the openAI completion connector to the kernel, choosing a name to identify it within the kernel
# after this instruction, the kernel has got it service_id that you can get with kernel.all_chat_services()

kernel.add_chat_service(
    "mauromi_chatgpt", # unique name to be registered with the kernel
    chat_connector
)
print(f"Here are all chat services registered with this kernel: {kernel.all_chat_services()}")

Here are all chat services registered with this kernel: ['mauromi_chatgpt']


# Create and Call Stand-Alone Semantic Functions

In [9]:
# Now that we have the chat service registered with the kernel, we can create and run a semantic function IN-LINE
# The semantic function is added to the list of kernel semantic functions
# We may also add an existing SEMANTIC plugin with kernel.import_semantic_plugin_from_directory(plugins_directory, "FunPlugin")
# We may also add an existing NATIVE plugin with import_native_plugin_from_directory(plugins_directory, "FunPlugin")
# https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

sk_prompt = "Give exactly 3 answers to the following question: {{$input}}"

# The following instruction adds a semantic function to the kernel. It requires that the kernel contains the chat or 
# text completion service, otherwise we get the error <TextCompletionClientBase service with service_id 'None' not found>
my_semantic_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    plugin_name="GenericSemanticPlugin",
    function_name="GenericSemanticFunction",
    description="Generic Semantic Function in a Generic Semantic Plugin",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

Here are my plugins imported into the kernel:
data={'catplugin': {'catage': KernelFunction(), 'catbirthday': KernelFunction()}, 'randomnumberplugin': {'generatenumberthreeorhigher': KernelFunction(), 'generatenumberthreeorlower': KernelFunction()}, 'genericsemanticplugin': {'genericsemanticfunction': KernelFunction()}}


In [10]:
# Call the semantic function directly, without extracting it from the kernel
# Note: the value of the "input" variable is replaced with the answer that we get from the semantic function

context = kernel.create_new_context()
context.variables["input"] = "Give me a wild animal"
print(f"Input Context Variable (before):\n<<<{context.variables.input}>>>")
print(f"Result:\n{my_semantic_function.invoke(context=context)}")
print(f"\nInput Context Variable (after):\n<<<{context.variables.input}>>>")

Input Context Variable (before):
<<<Give me a wild animal>>>
Result:
1. Lion
2. Tiger
3. Elephant

Input Context Variable (after):
<<<1. Lion
2. Tiger
3. Elephant>>>


In [11]:
# Same result as previous cell, but in this case we extract the semantic function from the kernel and THEN we invoke it

sf = kernel.plugins.get_function(plugin_name="GenericSemanticPlugin", function_name="GenericSemanticFunction")

context = kernel.create_new_context()
context.variables["input"] = "Who is Joe Biden's Wife?"

print(f"Input Context Variable (before):\n<<<{context.variables.input}>>>")
print(f"Result:\n{my_semantic_function.invoke(context=context)}")
print(f"\nInput Context Variable (after):\n<<<{context.variables.input}>>>")

Input Context Variable (before):
<<<Who is Joe Biden's Wife?>>>
Result:
1. Jill Biden is Joe Biden's wife.
2. Dr. Jill Biden is Joe Biden's wife.
3. Jill Tracy Jacobs Biden is Joe Biden's wife.

Input Context Variable (after):
<<<1. Jill Biden is Joe Biden's wife.
2. Dr. Jill Biden is Joe Biden's wife.
3. Jill Tracy Jacobs Biden is Joe Biden's wife.>>>


## Recap: which plugins and functions do we have of which type (native / semantic)?

In [12]:
import json

all_functions = kernel.plugins.get_functions_view()
json_object = json.loads(all_functions.json())
json_formatted_str = json.dumps(json_object, indent=2)

print(json_formatted_str)

{
  "semantic_functions": {
    "GenericSemanticPlugin": [
      {
        "name": "GenericSemanticFunction",
        "plugin_name": "GenericSemanticPlugin",
        "description": "Generic Semantic Function in a Generic Semantic Plugin",
        "is_semantic": true,
        "parameters": [
          {
            "name": "input",
            "description": "",
            "default_value": "",
            "type_": "string",
            "required": false
          }
        ],
        "is_asynchronous": true
      }
    ]
  },
  "native_functions": {
    "CatPlugin": [
      {
        "name": "CatAge",
        "plugin_name": "CatPlugin",
        "description": "Given a cat name, returns its age.",
        "is_semantic": false,
        "parameters": [
          {
            "name": "cat_name",
            "description": "The cat name",
            "default_value": "AA",
            "type_": "string",
            "required": false
          }
        ],
        "is_asynchronous": true
  

# Calling Native Functions from within a Semantic Function
One neat thing about the Semantic Kernel is that you can also call native functions from within Semantic Functions!

We will make our CorgiStory semantic function call a native function GenerateNames which will return names for our Corgi characters.

We do this using the syntax `{{plugin_name.function_name}}` or `{{plugin_name.function_name $param_name}}` or `{{plugin_name.function_name "param_value"}}`.You can read more about our prompte templating syntax [here](https://learn.microsoft.com/en-us/semantic-kernel/prompts/prompt-template-syntax).

In [13]:
# Now that we have the chat service registered with the kernel, we can create and run a semantic function IN-LINE
# The semantic function is added to the list of kernel semantic functions
# We may also add an existing SEMANTIC plugin with kernel.import_semantic_plugin_from_directory(plugins_directory, "FunPlugin")
# We may also add an existing NATIVE plugin with import_native_plugin_from_directory(plugins_directory, "FunPlugin")
# https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

# sk_prompt = 'Give exactly {"function": "CatPlugin.CatAge", "args": {"cat_name": "Molly"}} answers to the following question: {{$input}}'
sk_prompt_nested = 'Give exactly {{CatPlugin.CatAge $cat_name}} answers to the following question: {{$input}}'

# The following instruction adds a semantic function to the kernel. It requires that the kernel contains the chat or 
# text completion service, otherwise we get the error <TextCompletionClientBase service with service_id 'None' not found>
my_semantic_function_nested = kernel.create_semantic_function(
    prompt_template=sk_prompt_nested,
    plugin_name="GenericSemanticPluginNested",
    function_name="GenericSemanticFunctionNested",
    description="Generic Semantic FunctionNested in a Generic Semantic Plugin",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

Here are my plugins imported into the kernel:
data={'catplugin': {'catage': KernelFunction(), 'catbirthday': KernelFunction()}, 'randomnumberplugin': {'generatenumberthreeorhigher': KernelFunction(), 'generatenumberthreeorlower': KernelFunction()}, 'genericsemanticplugin': {'genericsemanticfunction': KernelFunction()}, 'genericsemanticpluginnested': {'genericsemanticfunctionnested': KernelFunction()}}


In [14]:
sf_nested = kernel.plugins.get_function(
    plugin_name="GenericSemanticPluginNested", 
    function_name="GenericSemanticFunctionNested"
)

context = kernel.create_new_context()
context.variables["input"] = "Tell me a kind or mammal"
context.variables["cat_name"] = "Molly"

print(f"Input Context Variable (before):\n<<<{context.variables.input}>>>")
print(f"Result:\n{sf_nested.invoke(context=context)}")
print(f"\nInput Context Variable (after):\n<<<{context.variables.input}>>>")

Input Context Variable (before):
<<<Tell me a kind or mammal>>>
Result:
1. Dog
2. Cat
3. Elephant
4. Dolphin
5. Giraffe

Input Context Variable (after):
<<<1. Dog
2. Cat
3. Elephant
4. Dolphin
5. Giraffe>>>


In [15]:
import json

all_functions = kernel.plugins.get_functions_view()
json_object = json.loads(all_functions.json())
json_formatted_str = json.dumps(json_object, indent=2)

print(json_formatted_str)

{
  "semantic_functions": {
    "GenericSemanticPlugin": [
      {
        "name": "GenericSemanticFunction",
        "plugin_name": "GenericSemanticPlugin",
        "description": "Generic Semantic Function in a Generic Semantic Plugin",
        "is_semantic": true,
        "parameters": [
          {
            "name": "input",
            "description": "",
            "default_value": "",
            "type_": "string",
            "required": false
          }
        ],
        "is_asynchronous": true
      }
    ],
    "GenericSemanticPluginNested": [
      {
        "name": "GenericSemanticFunctionNested",
        "plugin_name": "GenericSemanticPluginNested",
        "description": "Generic Semantic FunctionNested in a Generic Semantic Plugin",
        "is_semantic": true,
        "parameters": [
          {
            "name": "input",
            "description": "",
            "default_value": "",
            "type_": "string",
            "required": false
          }
      